## Importando bibliotecas e base

In [9]:
from warnings import filterwarnings
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import TunedThresholdClassifierCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, f1_score, make_scorer, accuracy_score, precision_score, recall_score, auc
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif
from util import *
import joblib
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx

filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [10]:
basket = pd.read_parquet("../data/processed/df_completo.parquet")
display(basket.head())

,Member_number,Date,itemDescription,transaction_by_member
0,1808,2015-07-21,tropical fruit,1808_2015-07-21
1,2552,2015-01-05,whole milk,2552_2015-01-05
2,2300,2015-09-19,pip fruit,2300_2015-09-19
3,1187,2015-12-12,other vegetables,1187_2015-12-12
4,3037,2015-02-01,whole milk,3037_2015-02-01


### Agrupamento em transações

In [23]:
basket.itemDescription = basket.itemDescription.transform(lambda x: [x])
basket = basket.groupby(['Member_number','Date']).sum()['itemDescription'].reset_index(drop=True)

encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(basket).transform(basket), columns=encoder.columns_)
display(transactions.head())

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,female sanitary products,finished products,fish,flour,flower (seeds),flower soil/fertilizer,frankfurter,frozen chicken,frozen dessert,frozen fish,frozen fruits,frozen meals,frozen potato products,frozen vegetables,fruit/vegetable juice,grapes,hair spray,ham,hamburger meat,hard cheese,herbs,honey,house keeping products,hygiene articles,ice cream,instant coffee,jam,ketchup,kitchen towels,kitchen utensil,light bulbs,liqueur,liquor,liquor (appetizer),liver loaf,long life bakery product,make up remover,male cosmetics,margarine,mayonnaise,meat,meat spreads,misc. beverages,mustard,napkins,newspapers,nut snack,nuts/prunes,oil,onions,organic products,organic sausage,other vegetables,packaged fruit/vegetables,pasta,pastry,pet care,photo/film,pickled vegetables,pip fruit,popcorn,pork,pot plants,potato products,preservation products,processed cheese,prosecco,pudding powder,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rubbing alcohol,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,Fa

## Regras a priori e de associação

In [24]:
frequent_itemsets = apriori(transactions, min_support= 6/len(basket), use_colnames=True, max_len = 2)
rules = association_rules(frequent_itemsets, metric="lift",  min_threshold = 1.5)
display(rules.head())
print("Rules identified: ", len(rules))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(butter milk),(UHT-milk),0.017577,0.021386,0.000601,0.034221,1.600131,1.0,0.000226,1.013289,0.381761,0.015679,0.013115,0.031173
1,(UHT-milk),(butter milk),0.021386,0.017577,0.000601,0.028125,1.600131,1.0,0.000226,1.010854,0.383247,0.015679,0.010737,0.031173
2,(cream cheese ),(UHT-milk),0.023658,0.021386,0.000869,0.036723,1.717152,1.0,0.000363,1.015922,0.427761,0.019667,0.015672,0.038674
3,(UHT-milk),(cream cheese ),0.021386,0.023658,0.000869,0.040625,1.717152,1.0,0.000363,1.017685,0.426767,0.019667,0.017378,0.038674
4,(soda),(artif. sweetener),0.097106,0.001938,0.000468,0.004818,2.485725,1.0,0.000280,1.002893,0.661986,0.004746,0.002885,0.123098


Rules identified:  190


In [27]:
display(rules.sort_values(["support", "confidence","lift"],axis = 0, ascending = False).head(8))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
154,(napkins),(pastry),0.022121,0.051728,0.001738,0.078550,1.518529,1.0,0.000593,1.029109,0.349192,0.024096,0.028285,0.056071
155,(pastry),(napkins),0.051728,0.022121,0.001738,0.033592,1.518529,1.0,0.000593,1.011869,0.360095,0.024096,0.011730,0.056071
20,(beverages),(sausage),0.016574,0.060349,0.001537,0.092742,1.536764,1.0,0.000537,1.035704,0.355168,0.020390,0.034474,0.059106
21,(sausage),(beverages),0.060349,0.016574,0.001537,0.025471,1.536764,1.0,0.000537,1.009129,0.371714,0.020390,0.009046,0.059106
81,(specialty chocolate),(citrus fruit),0.015973,0.053131,0.001403,0.087866,1.653762,1.0,0.000555,1.038081,0.401735,0.020731,0.036684,0.057141
80,(citrus fruit),(specialty chocolate),0.053131,0.015973,0.001403,0.026415,1.653762,1.0,0.000555,1.010726,0.417500,0.020731,0.010612,0.057141
104,(flour),(tropical fruit),0.009757,0.067767,0.001069,0.109589,1.617141,1.0,0.000408,1.046969,0.385385,0.013986,0.044862,0.062684
171,(processed cheese),(root vegetables),0.010158,0.069572,0.001069,0.105263,1.513019,1.0,0.000363,1.039891,0.342549,0.013594,0.038360,0.060316


## Support (Suporte)

### O que mede: Frequência com que um conjunto de itens aparece em todas as transações.
Interpretação:
- Na 1ª tabela: (butter milk, UHT-milk) tem suporte 0.000601 → Apenas 0.06% das transações contêm ambos os itens.
- Na 2ª tabela: (napkins, pastry) tem suporte 0.001738 → 0.17% das transações incluem esses itens juntos.

## Confidence (Confiança)

### O que mede: Probabilidade de encontrar o item B na transação dado que A está presente.
Interpretação:
- Na 1ª tabela: Regra (butter milk → UHT-milk) tem confiança 0.034221 → Se butter milk é comprado, há 3.42% de chance de UHT-milk também ser comprado.
- Na 2ª tabela: Regra (beverages → sausage) tem confiança 0.092742 → 9.27% de chance de sausage ser comprado se beverages estiver na cesta.

## Lift (Elevação)

### O que mede: Quanto mais provável é a compra de B quando A é comprado.
Interpretação:
- Na 1ª tabela:  A e B estão positivamente correlacionados (ex.: lift = 1.6 na 1ª tabela indica 60% mais chance de compra conjunta do que aleatória).
- Na 2ª tabela: (soda → artif. sweetener) tem lift 2.485725 → A compra de soda aumenta a chance de artif. sweetener em 148.6%.